In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm
from sklearn.preprocessing import FunctionTransformer
from scipy import stats
import statsmodels.api as sm

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor, BaggingRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import SVR
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.neural_network import MLPRegressor
from scipy.stats import norm
import copy
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor, Pool, metrics, cv
import xgboost as xgb
from scipy.stats import gmean
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import xgboost
from xgboost import XGBRegressor


In [2]:
exclude_cols = ['index', 'startdate']
temporal_attrs = ['startdate',
 'year',
 'quarter',
 'month',
 'week',
 'dayofyear',
 'season',
 'day_of_year_sin',
 'day_of_year_cos',
 'week_sin',
 'week_cos',
 'month_sin',
 'month_cos',
 'season_sin',
 'season_cos',
 'quarter_sin',
 'quarter_cos']
loc_attrs = ['lat', 'lon', 'loc_group']
embedding_attrs = ['climateregions__climateregion']
target=["contest-tmp2m-14d__tmp2m"]

In [3]:
selected_cols = ['nmme-tmp2m-56w__cancm3',
 'contest-wind-h500-14d__wind-hgt-500',
 'contest-pevpr-sfc-gauss-14d__pevpr',
 'contest-slp-14d__slp',
 'contest-prwtr-eatm-14d__prwtr',
 'elevation__elevation',
 'lon',
 'lat',
 'contest-pres-sfc-gauss-14d__pres',
 'contest-wind-h100-14d__wind-hgt-100',
 'climateregions__climateregion',
 'nmme-tmp2m-34w__ccsm3',
 'dayofyear',
 'season_sin',
 'day_of_year_cos',
 'day_of_year_sin',
 'nmme-prate-34w__cfsv2',
 'contest-precip-14d__precip',
 'nmme-prate-34w__gfdlflora',
 'wind-hgt-850-2010-9',
 'month',
 'wind-hgt-10-2010-3',
 'nmme-prate-34w__nasa',
 'sst-2010-2',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'wind-uwnd-250-2010-7',
 'month_sin',
 'contest-rhum-sig995-14d__rhum',
 'nmme0-prate-56w__cfsv20',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'week_cos',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'contest-wind-h850-14d__wind-hgt-850',
 'wind-uwnd-925-2010-15',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'sst-2010-9',
 'wind-vwnd-250-2010-10',
 'nmme0-prate-56w__nasa0',
 'nmme-prate-34w__cancm3',
 'nmme0-prate-56w__ccsm30',
 'sst-2010-10',
 'sst-2010-5',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-18',
 'nmme0-prate-56w__gfdlflora0',
 'nmme0-prate-56w__gfdl0',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__ccsm30',
 'nmme0-prate-56w__cancm40',
 'wind-vwnd-250-2010-13',
 'nmme0-prate-34w__cfsv20',
 'nmme0-prate-34w__cancm30',
 'nmme0-prate-34w__gfdlflora0',
 'wind-uwnd-250-2010-14',
 'nmme-prate-34w__gfdl',
 'nmme0-prate-34w__gfdl0',
 'wind-vwnd-250-2010-1',
 'nmme0-prate-56w__cancm30',
 'nmme0-prate-34w__cancm40',
 'nmme-prate-34w__cancm4',
 'nmme-prate-56w__gfdlflorb',
 'nmme0-prate-34w__nasa0',
 'nmme0-tmp2m-34w__cancm30',
 'wind-uwnd-250-2010-15',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-10-2010-5',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__ccsm40',
 'nmme-prate-34w__ccsm3',
 'nmme0-prate-56w__ccsm40',
 'wind-hgt-500-2010-3',
 'wind-hgt-10-2010-9',
 'wind-vwnd-250-2010-11',
 'wind-vwnd-250-2010-16',
 'sst-2010-3',
 'sst-2010-4',
 'nmme-prate-56w__ccsm3',
 'wind-vwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-250-2010-8',
 'week',
 'wind-uwnd-925-2010-20',
 'sst-2010-7',
 'wind-vwnd-925-2010-11',
 'nmme-prate-56w__ccsm4',
 'startdate',
 'mei__nip',
 'wind-uwnd-925-2010-19',
 'mei__mei',
 'wind-hgt-500-2010-1',
 'wind-vwnd-925-2010-6',
 'nmme0-prate-56w__nmme0mean',
 'wind-uwnd-925-2010-13',
 'wind-hgt-850-2010-8',
 'wind-hgt-100-2010-8',
 'wind-hgt-10-2010-2',
 'wind-uwnd-925-2010-10',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-34w__nmme0mean',
 'mei__meirank',
 'wind-hgt-500-2010-4',
 'wind-vwnd-250-2010-4',
 'wind-uwnd-925-2010-9',
 'wind-vwnd-250-2010-12',
 'wind-vwnd-250-2010-9',
 'wind-hgt-850-2010-10',
 'wind-hgt-850-2010-6',
 'wind-uwnd-925-2010-17',
 'sst-2010-8',
 'wind-hgt-100-2010-4',
 'sst-2010-1',
 'wind-vwnd-925-2010-17',
 'sst-2010-6',
 'wind-hgt-100-2010-2',
 'wind-hgt-10-2010-4',
 'wind-uwnd-925-2010-6',
 'wind-uwnd-250-2010-2',
 'wind-uwnd-925-2010-12',
 'wind-vwnd-250-2010-18',
 'wind-vwnd-925-2010-2',
 'wind-hgt-850-2010-7',
 'wind-uwnd-250-2010-12',
 'wind-hgt-100-2010-3',
 'wind-vwnd-250-2010-5',
 'wind-vwnd-925-2010-14',
 'wind-uwnd-925-2010-3',
 'wind-hgt-500-2010-5',
 'wind-vwnd-925-2010-16',
 'wind-uwnd-250-2010-20',
 'wind-uwnd-250-2010-6',
 'wind-vwnd-925-2010-18',
 'wind-hgt-100-2010-9',
 'wind-vwnd-925-2010-7',
 'wind-vwnd-250-2010-14',
 'wind-uwnd-250-2010-9']

In [4]:
removed_cols = ['wind-hgt-850-2010-4', 'wind-uwnd-250-2010-16']
diff_cols = ['contest-pevpr-sfc-gauss-14d__pevpr', 'nmme0-tmp2m-34w__cancm30', 'year']

In [5]:
spatial_df = pd.read_parquet('train.parquet', engine='pyarrow')

In [6]:
features = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3', 
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5',
 'diff_pres_1_first',
 'diff_pres_1_min',
 'diff_pres_1_max',
 'range_pres_1',
 'scale_pres_1',
 'diff_ccsm3_month_1_first',
 'diff_ccsm3_month_1_min',
 'diff_ccsm3_month_1_max',
 'range_ccsm3_month_1',
 'scale_ccsm3_month_1',
 'diff_sst_1_month_1_first',
 'diff_sst_1_month_1_min',
 'diff_sst_1_month_1_max',
 'range_sst_1_month_1',
 'scale_sst_1_month_1',
 'sst_1_lag_1',
 'sst_5_lag_1',
 'sst_4_lag_1',
 'sst_1_lag_1_season',
 'sst_5_lag_1_season',
 'sst_4_lag_1_season']

target=["contest-tmp2m-14d__tmp2m"]

slp_col = 'contest-slp-14d__slp'
ccsm3_col = 'nmme-tmp2m-34w__ccsm3'
pres_col = 'contest-pres-sfc-gauss-14d__pres'
precip_col = 'contest-precip-14d__precip'
poten_col = 'contest-pevpr-sfc-gauss-14d__pevpr'
rhum_col = 'contest-rhum-sig995-14d__rhum'
elevation_col = 'elevation__elevation'
wind_col_500 = 'contest-wind-h500-14d__wind-hgt-500'
sst_1_col = 'sst-2010-1'
sst_2_col = 'sst-2010-2'
sst_5_col = 'sst-2010-5'
sst_3_col = 'sst-2010-3'
sst_4_col = 'sst-2010-4'
sst_6_col = 'sst-2010-6'
sst_7_col = 'sst-2010-7'
sst_8_col = 'sst-2010-8'
sst_9_col = 'sst-2010-9'
sst_10_col = 'sst-2010-10'
group_cols = ['startdate']

In [8]:
spatial_cols = ['contest-wind-h500-14d__wind-hgt-500',
 'contest-slp-14d__slp',
 'nmme-tmp2m-34w__ccsm3',
 'elevation__elevation',
 'lon',
 'contest-prwtr-eatm-14d__prwtr',
 'lat',
 'climateregions__climateregion',
 'contest-pres-sfc-gauss-14d__pres',
 'season_sin',
 'day_of_year_sin',
 'contest-precip-14d__precip',
 'contest-wind-uwnd-250-14d__wind-uwnd-250',
 'nmme-prate-34w__cfsv2',
 'nmme-prate-34w__nasa',
 'nmme-prate-56w__gfdlflora',
 'wind-uwnd-250-2010-7',
 'contest-wind-vwnd-925-14d__wind-vwnd-925',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-34w__ccsm30',
 'contest-wind-h850-14d__wind-hgt-850',
 'contest-wind-uwnd-925-14d__wind-uwnd-925',
 'nmme0-prate-56w__cfsv20',
 'nmme-prate-34w__cancm3',
 'contest-rhum-sig995-14d__rhum',
 'nmme-prate-34w__gfdlflorb',
 'wind-hgt-850-2010-4',
 'contest-wind-vwnd-250-14d__wind-vwnd-250',
 'wind-hgt-100-2010-2',
 'wind-uwnd-250-2010-18',
 'wind-hgt-10-2010-5',
 'wind-uwnd-250-2010-15',
 'wind-uwnd-250-2010-4',
 'nmme0-prate-56w__nasa0',
 'nmme0-prate-34w__cfsv20',
 'wind-vwnd-250-2010-10',
 'contest-wind-h10-14d__wind-hgt-10',
 'wind-uwnd-925-2010-15',
 'wind-vwnd-250-2010-3',
 'nmme-prate-34w__cancm4',
 'sst-2010-4',
 'nmme0-prate-56w__ccsm30',
 'wind-uwnd-250-2010-16',
 'nmme0-prate-34w__gfdl0',
 'nmme0-prate-56w__cancm40',
 'sst-2010-1',
 'sst-2010-3',
 'wind-uwnd-250-2010-14',
 'nmme0-prate-34w__gfdlflora0',
 'nmme-prate-34w__gfdl',
 'wind-hgt-850-2010-9',
 'wind-vwnd-250-2010-1',
 'sst-2010-5',
 'cancm30',
 'nmme-prate-34w__ccsm4',
 'nmme0-prate-34w__nasa0',
 'wind-hgt-500-2010-9',
 'nmme0-prate-34w__cancm30',
 'wind-vwnd-250-2010-13',
 'wind_diff',
 'wind_diff_min',
 'wind_diff_min_month',
 'wind_diff_month',
 'diff_slp_first',
 'diff_wind_first',
 'diff_precip_first',
 'diff_sst_10_first',
 'diff_sst_10_min',
 'diff_sst_10_max',
 'range_sst_10',
 'scale_sst_10',
 'diff_sst_9_first',
 'diff_sst_9_min',
 'diff_sst_9_max',
 'range_sst_9',
 'scale_sst_9',
 'diff_sst_8_first',
 'diff_sst_8_min',
 'diff_sst_8_max',
 'range_sst_8',
 'scale_sst_8',
 'diff_sst_7_first',
 'diff_sst_7_min',
 'diff_sst_7_max',
 'range_sst_7',
 'scale_sst_7',
 'diff_sst_6_first',
 'diff_sst_6_min',
 'diff_sst_6_max',
 'range_sst_6',
 'scale_sst_6',
 'diff_sst_1_first',
 'diff_sst_1_min',
 'diff_sst_1_max',
 'range_sst_1',
 'scale_sst_1',
 'diff_sst_2_first',
 'diff_sst_2_min',
 'diff_sst_2_max',
 'range_sst_2',
 'scale_sst_2',
 'diff_sst_3_first',
 'diff_sst_3_min',
 'diff_sst_3_max',
 'range_sst_3',
 'scale_sst_3',
 'diff_sst_4_first',
 'diff_sst_4_min',
 'diff_sst_4_max',
 'range_sst_4',
 'scale_sst_4',
 'diff_sst_5_first',
 'diff_sst_5_min',
 'diff_sst_5_max',
 'range_sst_5',
 'scale_sst_5',
 'diff_pres_1_first',
 'diff_pres_1_min',
 'diff_pres_1_max',
 'range_pres_1',
 'scale_pres_1',
 'diff_ccsm3_month_1_first',
 'diff_ccsm3_month_1_min',
 'diff_ccsm3_month_1_max',
 'range_ccsm3_month_1',
 'scale_ccsm3_month_1',
 'diff_sst_1_month_1_first',
 'diff_sst_1_month_1_min',
 'diff_sst_1_month_1_max',
 'range_sst_1_month_1',
 'scale_sst_1_month_1',
 'sst_1_lag_1',
 'sst_5_lag_1',
 'sst_4_lag_1',
 'sst_1_lag_1_season',
 'sst_5_lag_1_season',
 'sst_4_lag_1_season',
 'sst_1_neigh_5_mean',
 'sst_1_neigh_5_min',
 'sst_1_neigh_5_max',
 'sst_5_neigh_5_mean',
 'sst_5_neigh_5_min',
 'sst_5_neigh_5_max',
 'sst_3_neigh_5_mean',
 'sst_3_neigh_5_min',
 'sst_3_neigh_5_max',
 'sst_4_neigh_5_mean',
 'sst_4_neigh_5_min',
 'sst_4_neigh_5_max',
 'wind_500_neigh_5_mean',
 'wind_500_neigh_5_min',
 'wind_500_neigh_5_max',
 'slp_neigh_5_mean',
 'slp_neigh_5_min',
 'slp_neigh_5_max',
 'precip_neigh_5_mean',
 'precip_neigh_5_min',
 'precip_neigh_5_max',
 'pres_neigh_5_mean',
 'pres_neigh_5_min',
 'pres_neigh_5_max',
 'rhum_neigh_5_mean',
 'rhum_neigh_5_min',
 'rhum_neigh_5_max',
 'slp_neigh_3_mean',
 'slp_neigh_3_min',
 'slp_neigh_3_max',
 'precip_neigh_3_mean',
 'precip_neigh_3_min',
 'precip_neigh_3_max',
 'pres_neigh_3_mean',
 'pres_neigh_3_min',
 'pres_neigh_3_max',
 'slp_neigh_7_mean',
 'slp_neigh_7_min',
 'slp_neigh_7_max',
 'rhum_neigh_7_mean',
 'rhum_neigh_7_min',
 'rhum_neigh_7_max',
 'sst_1_neigh_7_mean',
 'sst_1_neigh_7_min',
 'sst_1_neigh_7_max',
 'sst_5_neigh_7_mean',
 'sst_5_neigh_7_min',
 'sst_5_neigh_7_max',
 'sst_3_neigh_7_mean',
 'sst_3_neigh_7_min',
 'sst_3_neigh_7_max',
 'sst_4_neigh_7_mean',
 'sst_4_neigh_7_min',
 'sst_4_neigh_7_max',
 'precip_neigh_7_mean',
 'precip_neigh_7_min',
 'precip_neigh_7_max',
 'pres_neigh_7_mean',
 'pres_neigh_7_min',
 'pres_neigh_7_max',
 'wind_500_neigh_7_mean',
 'wind_500_neigh_7_min',
 'wind_500_neigh_7_max',
 'slp_neigh_9_mean',
 'slp_neigh_9_min',
 'slp_neigh_9_max',
 'wind_500_neigh_9_mean',
 'wind_500_neigh_9_min',
 'wind_500_neigh_9_max',
 'pres_neigh_9_mean',
 'pres_neigh_9_min',
 'pres_neigh_9_max',
 'precip_neigh_9_mean',
 'precip_neigh_9_min',
 'precip_neigh_9_max',
 'rhum_neigh_9_mean',
 'rhum_neigh_9_min',
 'rhum_neigh_9_max',
 'rhum_neigh_13_mean',
 'rhum_neigh_13_min',
 'rhum_neigh_13_max',
 'rhum_time_7_mean',
 'rhum_time_7_min',
 'rhum_time_7_max',
 'pres_time_7_mean',
 'pres_time_7_min',
 'pres_time_7_max',
 'precip_time_7_mean',
 'precip_time_7_min',
 'precip_time_7_max',
 'rhum_time_14_mean',
 'rhum_time_14_min',
 'rhum_time_14_max',
 'pres_time_14_mean',
 'pres_time_14_min',
 'pres_time_14_max',
 'precip_time_14_mean',
 'precip_time_14_min',
 'precip_time_14_max',
 'sst_1_neigh_5_lat_mean',
 'sst_1_neigh_5_lat_min',
 'sst_1_neigh_5_lat_max',
 'sst_5_neigh_5_lat_mean',
 'sst_5_neigh_5_lat_min',
 'sst_5_neigh_5_lat_max',
 'sst_3_neigh_5_lat_mean',
 'sst_3_neigh_5_lat_min',
 'sst_3_neigh_5_lat_max',
 'sst_4_neigh_5_lat_mean',
 'sst_4_neigh_5_lat_min',
 'sst_4_neigh_5_lat_max',
 'wind_500_neigh_5_lat_mean',
 'wind_500_neigh_5_lat_min',
 'wind_500_neigh_5_lat_max',
 'slp_neigh_5_lat_mean',
 'slp_neigh_5_lat_min',
 'slp_neigh_5_lat_max',
 'precip_neigh_5_lat_mean',
 'precip_neigh_5_lat_min',
 'precip_neigh_5_lat_max',
 'pres_neigh_5_lat_mean',
 'pres_neigh_5_lat_min',
 'pres_neigh_5_lat_max',
 'rhum_neigh_5_lat_mean',
 'rhum_neigh_5_lat_min',
 'rhum_neigh_5_lat_max',
 'sst_1_neigh_5_lon_mean',
 'sst_1_neigh_5_lon_min',
 'sst_1_neigh_5_lon_max',
 'sst_5_neigh_5_lon_mean',
 'sst_5_neigh_5_lon_min',
 'sst_5_neigh_5_lon_max',
 'sst_3_neigh_5_lon_mean',
 'sst_3_neigh_5_lon_min',
 'sst_3_neigh_5_lon_max',
 'sst_4_neigh_5_lon_mean',
 'sst_4_neigh_5_lon_min',
 'sst_4_neigh_5_lon_max',
 'wind_500_neigh_5_lon_mean',
 'wind_500_neigh_5_lon_min',
 'wind_500_neigh_5_lon_max',
 'slp_neigh_5_lon_mean',
 'slp_neigh_5_lon_min',
 'slp_neigh_5_lon_max',
 'precip_neigh_5_lon_mean',
 'precip_neigh_5_lon_min',
 'precip_neigh_5_lon_max',
 'pres_neigh_5_lon_mean',
 'pres_neigh_5_lon_min',
 'pres_neigh_5_lon_max',
 'rhum_neigh_5_lon_mean',
 'rhum_neigh_5_lon_min',
 'rhum_neigh_5_lon_max',
 'slp_neigh_5_diag_pos_mean',
 'slp_neigh_5_diag_pos_min',
 'slp_neigh_5_diag_pos_max',
 'precip_neigh_5_diag_pos_mean',
 'precip_neigh_5_diag_pos_min',
 'precip_neigh_5_diag_pos_max',
 'pres_neigh_5_diag_pos_mean',
 'pres_neigh_5_diag_pos_min',
 'pres_neigh_5_diag_pos_max',
 'rhum_neigh_5_diag_pos_mean',
 'rhum_neigh_5_diag_pos_min',
 'rhum_neigh_5_diag_pos_max']

In [9]:
# best_cols = list(set([c for c in ([59:] + selected_cols) if c not in (removed_cols + diff_cols)]))
best_cols = spatial_cols

In [10]:
spatial_df['type'] = ['train' for i in range(375734)] + ['test' for i in range(len(spatial_df) - 375734)]

In [11]:
train_df = spatial_df[spatial_df['type'] == 'train']
test_df = spatial_df[spatial_df['type'] == 'test']

In [12]:
train_df['startdate'].astype('str').unique().tolist()

['2014-09-01',
 '2014-09-02',
 '2014-09-03',
 '2014-09-04',
 '2014-09-05',
 '2014-09-06',
 '2014-09-07',
 '2014-09-08',
 '2014-09-09',
 '2014-09-10',
 '2014-09-11',
 '2014-09-12',
 '2014-09-13',
 '2014-09-14',
 '2014-09-15',
 '2014-09-16',
 '2014-09-17',
 '2014-09-18',
 '2014-09-19',
 '2014-09-20',
 '2014-09-21',
 '2014-09-22',
 '2014-09-23',
 '2014-09-24',
 '2014-09-25',
 '2014-09-26',
 '2014-09-27',
 '2014-09-28',
 '2014-09-29',
 '2014-09-30',
 '2014-10-01',
 '2014-10-02',
 '2014-10-03',
 '2014-10-04',
 '2014-10-05',
 '2014-10-06',
 '2014-10-07',
 '2014-10-08',
 '2014-10-09',
 '2014-10-10',
 '2014-10-11',
 '2014-10-12',
 '2014-10-13',
 '2014-10-14',
 '2014-10-15',
 '2014-10-16',
 '2014-10-17',
 '2014-10-18',
 '2014-10-19',
 '2014-10-20',
 '2014-10-21',
 '2014-10-22',
 '2014-10-23',
 '2014-10-24',
 '2014-10-25',
 '2014-10-26',
 '2014-10-27',
 '2014-10-28',
 '2014-10-29',
 '2014-10-30',
 '2014-10-31',
 '2014-11-01',
 '2014-11-02',
 '2014-11-03',
 '2014-11-04',
 '2014-11-05',
 '2014-11-

In [21]:

def train_test_by_df(train_df_, test_df_, iterations):
  X = train_df_[cols].values
  y = train_df_[target[0]].values

  skf = KFold(n_splits=5, random_state=2048, shuffle=True)
  skf.get_n_splits(X, y)

  train_index_list = []
  test_index_list = []
  train_x_list = []
  val_x_list = []
  train_y_list = []
  val_y_list = []

  for i, (train_index, test_index) in tqdm(enumerate(skf.split(X, y))):
    train_index_list.append(train_index)
    test_index_list.append(test_index)

    train_x_list.append(X[train_index, :])
    val_x_list.append(X[test_index, :])
    train_y_list.append(y[train_index])
    val_y_list.append(y[test_index]) 
  
    
  cat_models = []
  for i in tqdm(range(5)):
    train_x, val_x = train_x_list[i], val_x_list[i]
    train_y, val_y = train_y_list[i], val_y_list[i]
    clf = CatBoostRegressor( iterations = iterations, verbose = 50, loss_function='RMSEWithUncertainty', posterior_sampling=True,)
    clf.fit(train_x, train_y, eval_set=[(val_x, val_y)])
    cat_models.append(clf)

  preds = []
  for cat_model in tqdm(cat_models):
      preds.append(cat_model.predict(test_df_[cols].values))
    
  return preds, cat_models


In [ ]:
ens_preds, cat_models = ensemble(train_pool, val_pool, num_samples=10, iters=5000)

0:	learn: 3.6310135	test: 3.9792695	best: 3.9792695 (0)	total: 74.9ms	remaining: 6m 14s
10:	learn: 3.2867590	test: 3.6227382	best: 3.6227382 (10)	total: 840ms	remaining: 6m 21s
20:	learn: 3.0546113	test: 3.3789391	best: 3.3789391 (20)	total: 1.62s	remaining: 6m 24s
30:	learn: 2.8680613	test: 3.1749056	best: 3.1749056 (30)	total: 2.36s	remaining: 6m 18s
40:	learn: 2.7001789	test: 2.9769484	best: 2.9769484 (40)	total: 3.09s	remaining: 6m 13s
50:	learn: 2.5577731	test: 2.8032484	best: 2.8032484 (50)	total: 3.81s	remaining: 6m 10s
60:	learn: 2.4278424	test: 2.6359119	best: 2.6359119 (60)	total: 4.53s	remaining: 6m 6s
70:	learn: 2.3133736	test: 2.4800499	best: 2.4800499 (70)	total: 5.28s	remaining: 6m 6s
80:	learn: 2.2197016	test: 2.3650458	best: 2.3650458 (80)	total: 5.99s	remaining: 6m 3s
90:	learn: 2.1465249	test: 2.2715140	best: 2.2715140 (90)	total: 6.66s	remaining: 5m 59s
100:	learn: 2.0876773	test: 2.2023512	best: 2.2023512 (100)	total: 7.37s	remaining: 5m 57s
110:	learn: 2.0371179	t

910:	learn: 1.1414775	test: 1.5852620	best: 1.5742232 (790)	total: 1m 4s	remaining: 4m 47s
920:	learn: 1.1366266	test: 1.5848890	best: 1.5742232 (790)	total: 1m 4s	remaining: 4m 46s
930:	learn: 1.1324660	test: 1.5905814	best: 1.5742232 (790)	total: 1m 5s	remaining: 4m 46s
940:	learn: 1.1277409	test: 1.5922837	best: 1.5742232 (790)	total: 1m 6s	remaining: 4m 45s
950:	learn: 1.1216071	test: 1.5922092	best: 1.5742232 (790)	total: 1m 6s	remaining: 4m 44s
960:	learn: 1.1160367	test: 1.5935334	best: 1.5742232 (790)	total: 1m 7s	remaining: 4m 44s
970:	learn: 1.1111725	test: 1.5949692	best: 1.5742232 (790)	total: 1m 8s	remaining: 4m 43s
980:	learn: 1.1065687	test: 1.5939949	best: 1.5742232 (790)	total: 1m 9s	remaining: 4m 42s
990:	learn: 1.1017481	test: 1.5901810	best: 1.5742232 (790)	total: 1m 9s	remaining: 4m 41s
1000:	learn: 1.0971169	test: 1.5912149	best: 1.5742232 (790)	total: 1m 10s	remaining: 4m 41s
1010:	learn: 1.0927805	test: 1.5910392	best: 1.5742232 (790)	total: 1m 11s	remaining: 4m

1800:	learn: 0.8498652	test: 1.8132781	best: 1.5742232 (790)	total: 2m 4s	remaining: 3m 40s
1810:	learn: 0.8476720	test: 1.8235449	best: 1.5742232 (790)	total: 2m 4s	remaining: 3m 39s
1820:	learn: 0.8456284	test: 1.8254057	best: 1.5742232 (790)	total: 2m 5s	remaining: 3m 38s
1830:	learn: 0.8435785	test: 1.8290973	best: 1.5742232 (790)	total: 2m 5s	remaining: 3m 38s
1840:	learn: 0.8414872	test: 1.8312114	best: 1.5742232 (790)	total: 2m 6s	remaining: 3m 37s
1850:	learn: 0.8395923	test: 1.8369917	best: 1.5742232 (790)	total: 2m 7s	remaining: 3m 36s
1860:	learn: 0.8375800	test: 1.8394644	best: 1.5742232 (790)	total: 2m 7s	remaining: 3m 35s
1870:	learn: 0.8355814	test: 1.8432698	best: 1.5742232 (790)	total: 2m 8s	remaining: 3m 34s
1880:	learn: 0.8338239	test: 1.8468570	best: 1.5742232 (790)	total: 2m 9s	remaining: 3m 34s
1890:	learn: 0.8318843	test: 1.8477797	best: 1.5742232 (790)	total: 2m 9s	remaining: 3m 33s
1900:	learn: 0.8296575	test: 1.8500043	best: 1.5742232 (790)	total: 2m 10s	remai

2690:	learn: 0.6977872	test: 2.1976449	best: 1.5742232 (790)	total: 3m 2s	remaining: 2m 36s
2700:	learn: 0.6966516	test: 2.1991785	best: 1.5742232 (790)	total: 3m 3s	remaining: 2m 35s
2710:	learn: 0.6955117	test: 2.2023823	best: 1.5742232 (790)	total: 3m 3s	remaining: 2m 35s
2720:	learn: 0.6944249	test: 2.2070884	best: 1.5742232 (790)	total: 3m 4s	remaining: 2m 34s
2730:	learn: 0.6933744	test: 2.2095568	best: 1.5742232 (790)	total: 3m 5s	remaining: 2m 33s
2740:	learn: 0.6921298	test: 2.2132527	best: 1.5742232 (790)	total: 3m 5s	remaining: 2m 33s
2750:	learn: 0.6908067	test: 2.2177947	best: 1.5742232 (790)	total: 3m 6s	remaining: 2m 32s
2760:	learn: 0.6893814	test: 2.2384862	best: 1.5742232 (790)	total: 3m 7s	remaining: 2m 31s
2770:	learn: 0.6879581	test: 2.2588442	best: 1.5742232 (790)	total: 3m 7s	remaining: 2m 31s
2780:	learn: 0.6865770	test: 2.2654712	best: 1.5742232 (790)	total: 3m 8s	remaining: 2m 30s
2790:	learn: 0.6853732	test: 2.2673014	best: 1.5742232 (790)	total: 3m 9s	remain

3580:	learn: 0.5919425	test: 2.5533124	best: 1.5742232 (790)	total: 3m 59s	remaining: 1m 35s
3590:	learn: 0.5909766	test: 2.5626446	best: 1.5742232 (790)	total: 4m	remaining: 1m 34s
3600:	learn: 0.5900373	test: 2.5678597	best: 1.5742232 (790)	total: 4m 1s	remaining: 1m 33s
3610:	learn: 0.5890616	test: 2.5721432	best: 1.5742232 (790)	total: 4m 1s	remaining: 1m 33s
3620:	learn: 0.5881664	test: 2.5758809	best: 1.5742232 (790)	total: 4m 2s	remaining: 1m 32s
3630:	learn: 0.5871936	test: 2.5790781	best: 1.5742232 (790)	total: 4m 3s	remaining: 1m 31s
3640:	learn: 0.5863314	test: 2.5818978	best: 1.5742232 (790)	total: 4m 3s	remaining: 1m 30s
3650:	learn: 0.5854505	test: 2.5842906	best: 1.5742232 (790)	total: 4m 4s	remaining: 1m 30s
3660:	learn: 0.5846023	test: 2.5893693	best: 1.5742232 (790)	total: 4m 4s	remaining: 1m 29s
3670:	learn: 0.5838299	test: 2.5913584	best: 1.5742232 (790)	total: 4m 5s	remaining: 1m 28s
3680:	learn: 0.5830348	test: 2.5933901	best: 1.5742232 (790)	total: 4m 6s	remainin

...

4300:	learn: 0.5145453	test: 2.9378870	best: 1.5770604 (806)	total: 4m 40s	remaining: 45.5s
4310:	learn: 0.5136121	test: 2.9409575	best: 1.5770604 (806)	total: 4m 40s	remaining: 44.9s
4320:	learn: 0.5126265	test: 2.9446170	best: 1.5770604 (806)	total: 4m 41s	remaining: 44.2s
4330:	learn: 0.5117816	test: 2.9494970	best: 1.5770604 (806)	total: 4m 42s	remaining: 43.6s
4340:	learn: 0.5109333	test: 2.9529018	best: 1.5770604 (806)	total: 4m 42s	remaining: 42.9s
4350:	learn: 0.5101155	test: 2.9564525	best: 1.5770604 (806)	total: 4m 43s	remaining: 42.3s
4360:	learn: 0.5091545	test: 2.9563846	best: 1.5770604 (806)	total: 4m 44s	remaining: 41.6s
4370:	learn: 0.5082553	test: 2.9544919	best: 1.5770604 (806)	total: 4m 44s	remaining: 41s
4380:	learn: 0.5074134	test: 2.9587092	best: 1.5770604 (806)	total: 4m 45s	remaining: 40.3s
4390:	learn: 0.5065275	test: 2.9644314	best: 1.5770604 (806)	total: 4m 45s	remaining: 39.7s
4400:	learn: 0.5057122	test: 2.9681373	best: 1.5770604 (806)	total: 4m 46s	remaini

190:	learn: 1.7994867	test: 1.8766935	best: 1.8766935 (190)	total: 13.4s	remaining: 5m 36s
200:	learn: 1.7761057	test: 1.8635902	best: 1.8635902 (200)	total: 14s	remaining: 5m 35s
210:	learn: 1.7550508	test: 1.8457212	best: 1.8457212 (210)	total: 14.7s	remaining: 5m 33s
220:	learn: 1.7352991	test: 1.8266584	best: 1.8266584 (220)	total: 15.4s	remaining: 5m 32s
230:	learn: 1.7163965	test: 1.8115009	best: 1.8115009 (230)	total: 16.1s	remaining: 5m 31s
240:	learn: 1.6992534	test: 1.7967901	best: 1.7967901 (240)	total: 16.7s	remaining: 5m 30s
250:	learn: 1.6822893	test: 1.7809266	best: 1.7809266 (250)	total: 17.4s	remaining: 5m 28s
260:	learn: 1.6651260	test: 1.7664382	best: 1.7664382 (260)	total: 18.1s	remaining: 5m 28s
270:	learn: 1.6510731	test: 1.7588409	best: 1.7588409 (270)	total: 18.8s	remaining: 5m 27s
280:	learn: 1.6369690	test: 1.7494303	best: 1.7494303 (280)	total: 19.5s	remaining: 5m 26s
290:	learn: 1.6233670	test: 1.7380375	best: 1.7380375 (290)	total: 20.1s	remaining: 5m 25s
3

1100:	learn: 1.0525520	test: 1.5998333	best: 1.5392818 (725)	total: 1m 15s	remaining: 4m 25s
1110:	learn: 1.0487153	test: 1.6065803	best: 1.5392818 (725)	total: 1m 15s	remaining: 4m 25s
1120:	learn: 1.0450111	test: 1.6144104	best: 1.5392818 (725)	total: 1m 16s	remaining: 4m 24s
1130:	learn: 1.0414023	test: 1.6204027	best: 1.5392818 (725)	total: 1m 17s	remaining: 4m 23s
1140:	learn: 1.0370197	test: 1.6207213	best: 1.5392818 (725)	total: 1m 17s	remaining: 4m 23s
1150:	learn: 1.0335534	test: 1.6205091	best: 1.5392818 (725)	total: 1m 18s	remaining: 4m 22s
1160:	learn: 1.0298047	test: 1.6198424	best: 1.5392818 (725)	total: 1m 19s	remaining: 4m 21s
1170:	learn: 1.0261501	test: 1.6229880	best: 1.5392818 (725)	total: 1m 19s	remaining: 4m 21s
1180:	learn: 1.0223046	test: 1.6247258	best: 1.5392818 (725)	total: 1m 20s	remaining: 4m 20s
1190:	learn: 1.0189376	test: 1.6271184	best: 1.5392818 (725)	total: 1m 21s	remaining: 4m 19s
1200:	learn: 1.0152512	test: 1.6277675	best: 1.5392818 (725)	total: 1m

1990:	learn: 0.8123774	test: 1.8547188	best: 1.5392818 (725)	total: 2m 14s	remaining: 3m 22s
2000:	learn: 0.8106778	test: 1.8548863	best: 1.5392818 (725)	total: 2m 14s	remaining: 3m 22s
2010:	learn: 0.8088634	test: 1.8599491	best: 1.5392818 (725)	total: 2m 15s	remaining: 3m 21s
2020:	learn: 0.8065675	test: 1.8629344	best: 1.5392818 (725)	total: 2m 16s	remaining: 3m 20s
2030:	learn: 0.8046395	test: 1.8662118	best: 1.5392818 (725)	total: 2m 16s	remaining: 3m 19s
2040:	learn: 0.8030109	test: 1.8673470	best: 1.5392818 (725)	total: 2m 17s	remaining: 3m 19s
2050:	learn: 0.8009488	test: 1.8704456	best: 1.5392818 (725)	total: 2m 18s	remaining: 3m 18s
2060:	learn: 0.7992331	test: 1.8684188	best: 1.5392818 (725)	total: 2m 18s	remaining: 3m 17s
2070:	learn: 0.7970288	test: 1.8723287	best: 1.5392818 (725)	total: 2m 19s	remaining: 3m 17s
2080:	learn: 0.7952171	test: 1.8767167	best: 1.5392818 (725)	total: 2m 20s	remaining: 3m 16s
2090:	learn: 0.7931864	test: 1.8873279	best: 1.5392818 (725)	total: 2m

2880:	learn: 0.6665450	test: 2.2550094	best: 1.5392818 (725)	total: 3m 29s	remaining: 2m 34s
2890:	learn: 0.6653091	test: 2.2610133	best: 1.5392818 (725)	total: 3m 30s	remaining: 2m 33s
2900:	learn: 0.6641217	test: 2.2610237	best: 1.5392818 (725)	total: 3m 31s	remaining: 2m 33s
2910:	learn: 0.6627949	test: 2.2648399	best: 1.5392818 (725)	total: 3m 33s	remaining: 2m 32s
2920:	learn: 0.6615042	test: 2.2673532	best: 1.5392818 (725)	total: 3m 34s	remaining: 2m 32s
2930:	learn: 0.6602017	test: 2.2719905	best: 1.5392818 (725)	total: 3m 35s	remaining: 2m 31s
2940:	learn: 0.6587345	test: 2.2788427	best: 1.5392818 (725)	total: 3m 36s	remaining: 2m 31s
2950:	learn: 0.6573882	test: 2.2904324	best: 1.5392818 (725)	total: 3m 37s	remaining: 2m 30s
2960:	learn: 0.6559505	test: 2.2933636	best: 1.5392818 (725)	total: 3m 38s	remaining: 2m 30s
2970:	learn: 0.6545163	test: 2.2949631	best: 1.5392818 (725)	total: 3m 39s	remaining: 2m 30s
2980:	learn: 0.6531793	test: 2.2976107	best: 1.5392818 (725)	total: 3m